In [ ]:
# при установке pytorch geometric бывают трудности с пакетом torch-sparse нужно выбирать правильную версию cuda и pytorch
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-geometric
!pip install umap-learn

In [ ]:
from torch_geometric.nn import DeepGraphInfomax as DGI
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm
from torch_geometric.datasets import TUDataset
import torch.nn as nn
import numpy as np

In [ ]:
# классический датасет CORA
dataset = 'Cora'
path = './data'
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [ ]:
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

Так как граф задан целиком на входе, то можно использовать трансдуктивный подход к обучению DGI, если в графе появляются новые вершины и связи в течение времени - то нужен индуктивный вариант, там немного другая архитектура.
Нам нужен какой-нибудь энкодер, чтобы получить т.н. patch representations (которые потом будут использоваться для получения graph-level репрезентаций).
Мы берем все параметры, такие как Velickovic использовал в [статье](https://arxiv.org/pdf/1809.10341.pdf).
corruption function нужна, чтобы делать семплинг негативных примеров (делаем row-wise shuffling исходной матрицы), по сути corrupted graph состоит из тех же вершин, что оригинальный, но в нем вершины находятся в других местах. corruption function может быть какой-нибудь другой, например, можно порезать ребра в исходном графе.

Используйте различные архитектуры энкодера, чтобы получить лучший результат по метрике, опишите ваши подходы и оставьте вывод. Попробуйте **минимум 3** разных варианта. Каждый подход оценивается в 3 балла + 1 балл за визуализацию и интерпретацию.

Вы можете использовать предложенную визуализацию, либо реализовать другим способом и получить дополнительные 2 балла к заданию.
[Форма](https://forms.gle/q6NMQs3QLJQ48dut9) для отправки до 19.03 23:59 msk

In [ ]:
# здесь можно использовать любой энкодер - сравните разные по сложности слои
class Encoder(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
    # YOUR CODE HERE

    def forward(self, x, edge_index):
    # YOUR CODE HERE
        return x


In [ ]:
def corruption(x, edge_index):
    return x[torch.randperm(x.size(0))], edge_index

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DeepGraphInfomax(
    hidden_channels=512, encoder=Encoder(dataset.num_features, 512),
    summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
    corruption=corruption).to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(data.x, data.edge_index)
    loss = model.loss(pos_z, neg_z, summary)
    loss.backward()
    optimizer.step()
    return loss.item()


def test():
    model.eval()
    z, _, _ = model(data.x, data.edge_index)
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask], max_iter=150)
    return acc, z

In [ ]:
for epoch in range(1, 100): #желательно поставить побольше, например 300
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
acc, out = test()
print(f'Accuracy: {acc:.4f}')

In [ ]:
palette = {}

for n, y in enumerate(set(data.y.numpy())):
    palette[y] = f'C{n}'

In [ ]:
import umap.umap_ as umap
import seaborn as sns

In [ ]:
embd = umap.UMAP().fit_transform(out.detach().cpu().numpy())

In [ ]:
# UMAP plot после DGI
plt.figure(figsize=(10, 10))
sns.scatterplot(x=embd.T[0], y=embd.T[1], hue=data.y.cpu().numpy(), palette=palette)
plt.legend(bbox_to_anchor=(1,1), loc='upper left')
plt.savefig("umap_embd_dgi.png", dpi=120)

In [ ]:
embd_x = umap.UMAP().fit_transform(data.x.numpy())

In [ ]:
# исходник
plt.figure(figsize=(10, 10))
sns.scatterplot(x=embd_x.T[0], y=embd_x.T[1], hue=data.y.cpu().numpy(), palette=palette)
plt.legend(bbox_to_anchor=(1,1), loc='upper left')
plt.savefig("umap_embd.png", dpi=120)